In [1]:
!apt install swig cmake

The operation couldn’t be completed. Unable to locate a Java Runtime that supports apt.
Please visit http://www.java.com for information on installing Java.



In [2]:
!pip install -r https://raw.githubusercontent.com/huggingface/deep-rl-class/main/notebooks/unit1/requirements-unit1.txt

In [3]:

!sudo apt-get update
!sudo apt-get install -y python3-opengl
!apt install ffmpeg
!apt install xvfb
!pip3 install pyvirtualdisplay

Password:
sudo: a password is required
Password:

In [1]:
# Don't Run This
#import os
#os.kill(os.getpid(), 9)

In [3]:

# Virtual display
from pyvirtualdisplay import Display

virtual_display = Display(visible=0, size=(1400, 900))
virtual_display.start()

In [63]:
import gymnasium

from huggingface_sb3 import load_from_hub, package_to_hub
from huggingface_hub import notebook_login # To log to our Hugging Face account to be able to upload models to the Hub.

from stable_baselines3 import PPO
from stable_baselines3.common.env_util import make_vec_env
from stable_baselines3.common.evaluation import evaluate_policy
from stable_baselines3.common.monitor import Monitor

In [64]:
import gymnasium as gym
# We create our environment with gym.make("<name_of_the_environment>")
env = gym.make("LunarLander-v2")
env.reset()
print("_____OBSERVATION SPACE_____ \n")
print("Observation Space Shape", env.observation_space.shape)
print("Sample observation", env.observation_space.sample()) # Get a random observation

_____OBSERVATION SPACE_____ 

Observation Space Shape (8,)
Sample observation [  0.7721408  -64.39547      1.8324583    4.6403084   -2.7791383
   2.783428     0.3846506    0.60663104]


In [65]:
print("\n _____ACTION SPACE_____ \n")
print("Action Space Shape", env.action_space.n)
print("Action Space Sample", env.action_space.sample()) # Take a random action


 _____ACTION SPACE_____ 

Action Space Shape 4
Action Space Sample 2


In [96]:
# Create the environment
env = make_vec_env('LunarLander-v2', n_envs=32)

In [97]:
import torch

# Check if MPS is available and built
print(torch.backends.mps.is_available())
print(torch.backends.mps.is_built())

# Set the PyTorch device to use MPS
device = torch.device('mps')
print(device)

True
True
mps


In [99]:
import torch
# Define PPO model with MlpPolicy
n_steps = 8192
batch_size = 256
n_epochs = 8

# Create the model without explicitly setting the device
model = PPO(
    policy='MlpPolicy',
    env=env,
    n_steps=n_steps,
    batch_size=batch_size,
    n_epochs=n_epochs,
    gamma=0.99999,
    gae_lambda=0.95,
    ent_coef=0.02,
    verbose=1
)


Using cpu device


In [100]:
# SOLUTION
# Train it for 10,000,000 timesteps
model.learn(total_timesteps=10000000)
# Save the model
model_name = "ppo-LunarLander-v2"
model.save(model_name)

---------------------------------
| rollout/           |          |
|    ep_len_mean     | 93.5     |
|    ep_rew_mean     | -184     |
| time/              |          |
|    fps             | 10884    |
|    iterations      | 1        |
|    time_elapsed    | 24       |
|    total_timesteps | 262144   |
---------------------------------
----------------------------------------
| rollout/                |            |
|    ep_len_mean          | 94.7       |
|    ep_rew_mean          | -134       |
| time/                   |            |
|    fps                  | 8621       |
|    iterations           | 2          |
|    time_elapsed         | 60         |
|    total_timesteps      | 524288     |
| train/                  |            |
|    approx_kl            | 0.01097118 |
|    clip_fraction        | 0.128      |
|    clip_range           | 0.2        |
|    entropy_loss         | -1.38      |
|    explained_variance   | 0.000325   |
|    learning_rate        | 0.0003     |
|   

In [101]:
#!pip install matplotlib numpy

In [102]:
import matplotlib.pyplot as plt
import numpy as np

In [103]:
from stable_baselines3.common.monitor import Monitor
from stable_baselines3.common.evaluation import evaluate_policy

In [117]:
# Monitor Graph
eval_env = Monitor(gym.make("LunarLander-v2"))

# Initialize empty lists to store rewards
mean_rewards = []
std_rewards = []

# Number of evaluation episodes
n_eval_episodes = 10

# Total training timesteps
total_timesteps = 10000000

# Training and evaluating in chunks
chunk_size = 1000000

for t in range(0, total_timesteps, chunk_size):
    # Train the model for the current chunk
    model.learn(total_timesteps=chunk_size)

    # Evaluate policy for multiple episodes
    mean_reward, std_reward = evaluate_policy(model, eval_env, n_eval_episodes=n_eval_episodes, deterministic=True)

    # Append results to the lists
    mean_rewards.append(mean_reward)
    std_rewards.append(std_reward)

    print(f"Timestep {t + chunk_size}: mean_reward={mean_reward:.2f} +/- {std_reward}")

# Plotting the results
timesteps = np.arange(chunk_size, total_timesteps + 1, chunk_size)
plt.errorbar(timesteps, mean_rewards, yerr=std_rewards, fmt='o-', label='Mean Reward')
plt.xlabel('Training Timesteps')
plt.ylabel('Mean Reward +/- Std Dev')
plt.title('Training Progress')
plt.legend()
plt.show()

---------------------------------
| rollout/           |          |
|    ep_len_mean     | 983      |
|    ep_rew_mean     | 176      |
| time/              |          |
|    fps             | 2157     |
|    iterations      | 1        |
|    time_elapsed    | 121      |
|    total_timesteps | 262144   |
---------------------------------


In [116]:
#@title
eval_env = Monitor(gym.make("LunarLander-v2"))
mean_reward, std_reward = evaluate_policy(model, eval_env, n_eval_episodes=10, deterministic=True)
print(f"mean_reward={mean_reward:.2f} +/- {std_reward}")

mean_reward=298.56 +/- 16.367726640087895
